## Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [2]:
%matplotlib inline

## Example: USDA Food Database

The US Department of Agriculture makes available a database of food nutrient information. Ashley Williams, an English hacker, has made available a version of this database in JSON format (http://ashleyw.co.uk/project/food-nutrient-database). The records look like this:

In [1]:
import json
db = json.load(open('foods-2011-10-03.json'))
len(db)

6636

In [11]:
len(db)

6636

In [10]:
len(db[3]['nutrients'])

213

In [4]:
from pandas import DataFrame

In [5]:
DataFrame(db)

,description,group,id,manufacturer,nutrients,portions,tags
0,"Cheese, caraway",Dairy and Egg Products,1008,,"[{'value': 25.18, 'units': 'g', 'description':...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}]",[]
1,"Cheese, cheddar",Dairy and Egg Products,1009,,"[{'value': 24.9, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'cup, diced', 'grams': ...",[]
2,"Cheese, edam",Dairy and Egg Products,1018,,"[{'value': 4.22, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...",[]
3,"Cheese, feta",Dairy and Egg Products,1019,,"[{'value': 5.2, 'units': 'g', 'description': '...","[{'amount': 1, 'unit': 'cup, crumbled', 'grams...",[]
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,,"[{'value': 3.27, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}]",[]
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,,"[{'value': 3.72, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'cup, diced', 'grams': ...",[]
6,"Cheese, romano",Dairy and Egg Products,1038,,"[{'value': 31.8, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...",[]
7,"Cheese, roquefort",Dairy and Egg Products,1039,,"[{'value': 21.54, 'units': 'g', 'description':...","[{'amount': 1, 'unit': 'oz', 'grams': 28.35}, ...",[]
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,,"[{'value': 16.41, 'units': 'g', 'description':...","[{'amount': 1, 'unit': 'cup, diced', 'grams': ...",[]
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,,"[{'value': 2.96, 'units': 'g', 'description': ...","[{'amount': 1, 'unit': 'cup', 'grams': 242.0},...",[]


In [13]:
db[0]

{'id': 1008,
 'description': 'Cheese, caraway',
 'tags': [],
 'manufacturer': '',
 'group': 'Dairy and Egg Products',
 'portions': [{'amount': 1, 'unit': 'oz', 'grams': 28.35}],
 'nutrients': [{'value': 25.18,
   'units': 'g',
   'description': 'Protein',
   'group': 'Composition'},
  {'value': 29.2,
   'units': 'g',
   'description': 'Total lipid (fat)',
   'group': 'Composition'},
  {'value': 3.06,
   'units': 'g',
   'description': 'Carbohydrate, by difference',
   'group': 'Composition'},
  {'value': 3.28, 'units': 'g', 'description': 'Ash', 'group': 'Other'},
  {'value': 376.0,
   'units': 'kcal',
   'description': 'Energy',
   'group': 'Energy'},
  {'value': 39.28,
   'units': 'g',
   'description': 'Water',
   'group': 'Composition'},
  {'value': 1573.0, 'units': 'kJ', 'description': 'Energy', 'group': 'Energy'},
  {'value': 0.0,
   'units': 'g',
   'description': 'Fiber, total dietary',
   'group': 'Composition'},
  {'value': 673.0,
   'units': 'mg',
   'description': 'Calcium,

In [78]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [14]:
db[0]['nutrients']

[{'value': 25.18,
  'units': 'g',
  'description': 'Protein',
  'group': 'Composition'},
 {'value': 29.2,
  'units': 'g',
  'description': 'Total lipid (fat)',
  'group': 'Composition'},
 {'value': 3.06,
  'units': 'g',
  'description': 'Carbohydrate, by difference',
  'group': 'Composition'},
 {'value': 3.28, 'units': 'g', 'description': 'Ash', 'group': 'Other'},
 {'value': 376.0, 'units': 'kcal', 'description': 'Energy', 'group': 'Energy'},
 {'value': 39.28,
  'units': 'g',
  'description': 'Water',
  'group': 'Composition'},
 {'value': 1573.0, 'units': 'kJ', 'description': 'Energy', 'group': 'Energy'},
 {'value': 0.0,
  'units': 'g',
  'description': 'Fiber, total dietary',
  'group': 'Composition'},
 {'value': 673.0,
  'units': 'mg',
  'description': 'Calcium, Ca',
  'group': 'Elements'},
 {'value': 0.64,
  'units': 'mg',
  'description': 'Iron, Fe',
  'group': 'Elements'},
 {'value': 22.0,
  'units': 'mg',
  'description': 'Magnesium, Mg',
  'group': 'Elements'},
 {'value': 490.0,

In [22]:

# rows list intialization 
rows = [] 
product_keys= ['id','description','manufacturer','group']
nutrient_keys = ['value','units','description','group']
# appending rows 
for product in db: 
    print(product['description'])
    
    product_dict = dict((k, product[k]) for k in product_keys)
    
    nutrients = product['nutrients']  
      
    for nutrient in nutrients: 
        row_dict = product_dict.copy()
        for k in nutrient_keys:
            row_dict.update({'nutrient_'+k: nutrient[k]}) 
        rows.append(row_dict) 
  
# using data frame 
df = DataFrame(rows) 
  
# print(df) 

Cheese, caraway
Cheese, cheddar
Cheese, edam
Cheese, feta
Cheese, mozzarella, part skim milk
Cheese, mozzarella, part skim milk, low moisture
Cheese, romano
Cheese, roquefort
Cheese spread, pasteurized process, american, without di sodium phosphate
Cream, fluid, half and half
Sour dressing, non-butterfat, cultured, filled cream-type
Milk, filled, fluid, with blend of hydrogenated vegetable oils
Cream substitute, liquid, with lauric acid oil and sodium caseinate
Cream substitute, powdered
Milk, producer, fluid, 3.7% milkfat
Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D
Milk, reduced fat, fluid, 2% milkfat, with added nonfat milk solids and vitamin A and vitamin D
Milk, reduced fat, fluid, 2% milkfat, protein fortified, with added vitamin A and vitamin D
Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D
Milk, lowfat, fluid, 1% milkfat, with added nonfat milk solids, vitamin A and vitamin D
Milk, lowfat, fluid, 1% milkfat, protein fortified, wi

Beef sausage, pre-cooked
Turkey sausage, fresh, raw
Beef sausage, fresh, cooked
Pork and turkey sausage, pre-cooked
Turkey sausage, fresh, cooked
Bologna, chicken, pork, beef
Bologna, chicken, pork
Cereals ready-to-eat, KELLOGG, KELLOGG'S ALL-BRAN Original
Cereals ready-to-eat, POST, ALPHA-BITS
Cereals ready-to-eat, KELLOGG, KELLOGG'S APPLE JACKS
Cereals ready-to-eat, KELLOGG, KELLOGG'S ALL-BRAN BRAN BUDS
Cereals ready-to-eat, QUAKER, CAP'N CRUNCH
Cereals ready-to-eat, QUAKER, CAP'N CRUNCH with CRUNCHBERRIES
Cereals ready-to-eat, QUAKER, CAP'N CRUNCH'S PEANUT BUTTER CRUNCH
Cereals ready-to-eat, GENERAL MILLS, CHEERIOS
Cereals ready-to-eat, KELLOGG, KELLOGG'S COCOA KRISPIES
Cereals ready-to-eat, POST, COCOA PEBBLES
Cereals ready-to-eat, GENERAL MILLS, COOKIE CRISP
Cereals ready-to-eat, QUAKER, QUAKER CRUNCHY BRAN
Cereals ready-to-eat, GENERAL MILLS, Corn CHEX
Cereals ready-to-eat, KELLOGG, KELLOGG'S Corn Flakes
Cereals ready-to-eat, corn flakes, low sodium
Cereals ready-to-eat, KELLOGG,

Pork, fresh, loin, center loin (chops), bone-in, separable lean only, raw
Pork, fresh, loin, center loin (chops), bone-in, separable lean only, cooked, braised
Pork, fresh, loin, center loin (chops), bone-in, separable lean only, cooked, broiled
Pork, fresh, loin, center loin (roasts), bone-in, separable lean only, cooked, roasted
Pork, fresh, loin, center rib (chops or roasts), bone-in, separable lean and fat, raw
Pork, fresh, loin, center rib (chops), bone-in, separable lean and fat, cooked, braised
Pork, fresh, loin, center rib (chops), bone-in, separable lean and fat, cooked, broiled
Pork, fresh, loin, center rib (roasts), bone-in, separable lean and fat, cooked, roasted
Pork, fresh, loin, center rib (chops or roasts), bone-in, separable lean only, raw
Pork, fresh, loin, center rib (chops), bone-in, separable lean only, cooked, braised
Pork, fresh, loin, center rib (chops), bone-in, separable lean only, cooked, broiled
Pork, fresh, loin, center rib (roasts), bone-in, separable lean

Fish, salmon, pink, canned, drained solids with bone
Fish, tilapia, raw
Fish, tilapia, cooked, dry heat
Beans, baked, home prepared
Beans, kidney, all types, mature seeds, canned
Carob flour
Cowpeas, common (blackeyes, crowder, southern), mature seeds, canned with pork
Lentils, raw
Soybeans, mature seeds, raw
Miso
Natto
Tempeh
Soy meal, defatted, raw
Soymilk, original and vanilla, unfortified
Soy protein isolate
Tofu, okara
Hummus, home prepared
Falafel, home-prepared
Soymilk, original and vanilla, with added calcium, vitamins A and D
Lentils, pink, raw
Veggie burgers or soyburgers, unprepared
Peanut spread, reduced sugar
Peanut butter, smooth, reduced fat
ENSURE FIBER WITH FOS, liquid
Chickpea flour (besan)
Hummus, commercial
Tofu, extra firm, prepared with nigari
Tofu, hard, prepared with nigari
MORI-NU, Tofu, silken, soft
MORI-NU, Tofu, silken, firm
MORI-NU, Tofu, silken, extra firm
MORI-NU, Tofu, silken, lite firm
MORI-NU, Tofu, silken, lite extra firm
Soymilk, chocolate, unfortifi

Spaghetti, cooked, enriched, with added salt
Cornmeal, degermed, enriched, white
Cornmeal, self-rising, bolted, plain, enriched, white
Cornmeal, self-rising, bolted, with wheat flour added, enriched, white
Cornmeal, self-rising, degermed, enriched, white
Hominy, canned, yellow
Rice, white, long-grain, regular, cooked, enriched, with salt
Wheat flour, white, all-purpose, enriched, calcium-fortified
Macaroni, cooked, unenriched
Noodles, egg, dry, unenriched
Noodles, egg, cooked, unenriched, without added salt
Spaghetti, dry, unenriched
Spaghetti, cooked, unenriched, without added salt
Cornmeal, degermed, unenriched, yellow
Rice, white, long-grain, regular, raw, unenriched
Rice, white, long-grain, regular, cooked, unenriched, without salt
Rice, white, long-grain, parboiled, unenriched, dry
Rice, white, long-grain, parboiled, unenriched, cooked
Rice, white, medium-grain, raw, unenriched
Rice, white, medium-grain, cooked, unenriched
Rice, white, short-grain, raw, unenriched
Rice, white, sho

Peppers, sweet, green, frozen, chopped, unprepared
Pigeonpeas, immature seeds, raw
Pigeonpeas, immature seeds, cooked, boiled, drained, without salt
Pokeberry shoots, (poke), raw
Pokeberry shoots, (poke), cooked, boiled, drained, without salt
Potato, flesh and skin, raw
Potatoes, russet, flesh and skin, raw
Potatoes, white, flesh and skin, raw
Potatoes, red, flesh and skin, raw
Potatoes, Russet, flesh and skin, baked
Potatoes, white, flesh and skin, baked
Potatoes, red, flesh and skin, baked
Potatoes, french fried, crinkle or regular cut, salt added in processing, frozen, oven-heated
Potatoes, raw, skin
Potatoes, baked, flesh, without salt
Potatoes, baked, skin, without salt
Potatoes, boiled, cooked in skin, flesh, without salt
Potatoes, boiled, cooked in skin, skin, without salt
Potatoes, boiled, cooked without skin, flesh, without salt
Potatoes, microwaved, cooked in skin, flesh, without salt
Potatoes, microwaved, cooked in skin, skin, without salt
Potatoes, canned, solids and liquid

Tortillas, ready-to-bake or -fry, corn, without added salt
Tortillas, ready-to-bake or -fry, flour, without added calcium
Candies, NESTLE, BUTTERFINGER Bar
Candies, gumdrops, starch jelly pieces
Candies, KIT KAT Wafer Bar
Candies, milk chocolate, with rice cereal
Sugars, brown
Candies, semisweet chocolate, made with butter
Gelatin desserts, dry mix, reduced calorie, with aspartame, added phosphorus, potassium, sodium, vitamin C
Pizza, cheese topping, regular crust, frozen, cooked
Pizza, cheese topping, rising crust, frozen, cooked
Pizza, meat and vegetable topping, regular crust, frozen, cooked
Pizza, meat and vegetable topping, rising crust, frozen, cooked
WENDY'S, French Fries
WENDY'S, Frosty Dairy Dessert
McDONALD'S, Vanilla TRIPLE THICK Shake
McDONALD'S, Chocolate TRIPLE THICK Shake
McDONALD'S, Strawberry TRIPLE THICK Shake
KENTUCKY FRIED CHICKEN, Fried Chicken, EXTRA CRISPY, Wing, meat only, skin and breading removed
KENTUCKY FRIED CHICKEN, Fried Chicken, ORIGINAL RECIPE, Breast, 

In [24]:
df.shape

(389355, 8)

In [28]:
df.head(10)

,description,group,id,manufacturer,nutrient_description,nutrient_group,nutrient_units,nutrient_value
0,"Cheese, caraway",Dairy and Egg Products,1008,,Protein,Composition,g,25.18
1,"Cheese, caraway",Dairy and Egg Products,1008,,Total lipid (fat),Composition,g,29.20
2,"Cheese, caraway",Dairy and Egg Products,1008,,"Carbohydrate, by difference",Composition,g,3.06
3,"Cheese, caraway",Dairy and Egg Products,1008,,Ash,Other,g,3.28
4,"Cheese, caraway",Dairy and Egg Products,1008,,Energy,Energy,kcal,376.00
5,"Cheese, caraway",Dairy and Egg Products,1008,,Water,Composition,g,39.28
6,"Cheese, caraway",Dairy and Egg Products,1008,,Energy,Energy,kJ,1573.00
7,"Cheese, caraway",Dairy and Egg Products,1008,,"Fiber, total dietary",Composition,g,0.00
8,"Cheese, caraway",Dairy and Egg Products,1008,,"Calcium, Ca",Elements,mg,673.00
9,"Cheese, caraway",Dairy and Egg Products,1008,,"Iron, Fe",Elements,mg,0.64


In [29]:
df.group.unique()

array(['Dairy and Egg Products', 'Spices and Herbs', 'Baby Foods',
       'Fats and Oils', 'Poultry Products', 'Soups, Sauces, and Gravies',
       'Sausages and Luncheon Meats', 'Breakfast Cereals',
       'Fruits and Fruit Juices', 'Pork Products',
       'Vegetables and Vegetable Products', 'Nut and Seed Products',
       'Beef Products', 'Beverages', 'Finfish and Shellfish Products',
       'Legumes and Legume Products', 'Lamb, Veal, and Game Products',
       'Baked Products', 'Snacks', 'Sweets', 'Cereal Grains and Pasta',
       'Fast Foods', 'Meals, Entrees, and Sidedishes', 'Ethnic Foods',
       'Restaurant Foods'], dtype=object)

In [79]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [80]:
info[:5]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [81]:
info

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [82]:
pd.value_counts(info.group)[:10]

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Fast Foods                           365
Legumes and Legume Products          365
Lamb, Veal, and Game Products        345
Sweets                               341
Fruits and Fruit Juices              328
Pork Products                        328
Name: group, dtype: int64

In [83]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
result['Zinc, Zn'].sort_values().plot(kind='barh')

NameError: name 'ndata' is not defined